# Web Scraping

## Introdução
Neste notebook, vamos fazer o webscrapping das imagens dos personagens de anime que serão usadas para treinar e testar a IA reconhecedora, gerando assim nosso dataset.

## Sumário
1. [Instalação de Bibliotecas](#Instalação-de-Bibliotecas)
2. [Inicialização](#Inicialização)
3. [Obtenção dos Personagens](#Obtenção-dos-Personagens)
4. [Obtenção das Imagens]()

## Instalação de Bibliotecas

Primeiramente, se ainda não foi feito, instalaremos todas as bibliotecas necessárias para esse projeto:


In [ ]:
%pip install -r requirements.txt

## Inicialização

Agora importaremos as bibliotecas e também inicializaremos as constantes globais que serão usadas a seguir:

In [5]:
import requests
from bs4 import BeautifulSoup, ResultSet, Tag
from pathlib import Path

CHAR_FOLDER = Path("Characters")
TEST_FOLDER = CHAR_FOLDER / Path("Test")
TRAIN_FOLDER = CHAR_FOLDER / Path("Train")
CHAR_LIST = Path("characters.txt")

## Obtenção dos Personagens

Aqui iremos obter a lista de todos os personagens principais que iremos treinar nossa IA, no caso iremos escolher os personagens do arco do Stardust Crusaders do anime JoJo's Bizarre Adventure, além disso iremos estruturar os diretórios para o salvamento dos arquivos.

### Obtenção da Lista de Nomes

Aqui iremos criar um arquivo `characters.txt` com a lista dos nomes dos personagens principais, utilizando o site do MyAnimesList como referência:

In [25]:
def get_characters(url):
    response = requests.get(url)
    parsed_html = BeautifulSoup(response.content, "html.parser")
    characters: ResultSet[Tag] = parsed_html.find_all(
        "h3", class_="h3_character_name"
    )
    with open(CHAR_LIST, "w", encoding="utf-8") as character_file:
        for character in characters:
            div_parent: Tag = character.parent.parent.parent
            character_role: Tag = div_parent.findChildren("div")[3]
            
            # Verificamos se o personagem é um personagem principal
            if character_role.text.strip() == "Main":
                
                # Como o nome vem no padrão oriental iremos redefinir para o padrão ocidental, caso necessário
                character = character.text
                if ", " in character:
                    name_split = character.split(", ")
                    character = name_split[1] + " " + name_split[0]
                print(character)
                print(character, file=character_file)

url = "https://myanimelist.net/anime/26055/JoJo_no_Kimyou_na_Bouken_Part_3__Stardust_Crusaders_2nd_Season/characters"
get_characters(url)

Muhammad Avdol
Dio Brando
Iggy
Joseph Joestar
Noriaki Kakyouin
Joutarou Kuujou
Jean-Pierre Polnareff


### Estruturação das pastas

Agora iremos criar e estruturar as pastas, no qual iremos guardar e separar as imagens. Criando uma pasta para cada personagem.



In [28]:
CHAR_FOLDER.mkdir(exist_ok=True)
TEST_FOLDER.mkdir(exist_ok=True)
TRAIN_FOLDER.mkdir(exist_ok=True)
with open(CHAR_LIST, "r", encoding="utf-8") as characters:
    for character in characters:
        character_name = character.strip().replace(" ","-")
        pasta = TEST_FOLDER / character_name
        pasta.mkdir(exist_ok=True)
        pasta = TRAIN_FOLDER / character_name
        pasta.mkdir(exist_ok=True)